## Подключение к БД

Импортируем библиотеки

In [1]:
import psycopg2
import pandas as pd

Устанавливаем соединение с БД

In [54]:
try:
    psw = (open("/content/pgsqlpsw.txt", "r")).readlines()
    connection = psycopg2.connect(user = psw[0], # Имя пользователя
        password = psw[1], # Пароль для пользователя
        host = '255.255.255.255', # Адрес
        port = '5432', # Порт
        database = 'dbstud') # Имя БД
    print("Cоединение установлено.")
except Exception as e:
    print(f"Ошибка соединения: {e}")


Cоединение установлено.


In [21]:
cursor = connection.cursor()

## 1) Создадим excel таблицу для преподавателя кружка риссования со всеми его учениками

In [20]:
query = """Select \"ADD_EDUCATION_GROUP\".\"StudentId\", \"STUDENTS\".\"StudentName\" as \"Имя учащегося\", \"STUDENTS\".\"Class\" as \"Класс\"
from \"ADD_EDUCATION_GROUP\" join \"STUDENTS\" on \"ADD_EDUCATION_GROUP\".\"StudentId\" = \"STUDENTS\".\"StudentId\"
where \"NameEdu\" = 'Рисование' order by \"StudentName\""""

data = pd.read_sql(query, connection, index_col="StudentId") #получим датафрейм
data.to_excel("/content/sample_data/Кружок рисования - список.xlsx") #запишим данные в excel таблицу
data

,Имя учащегося,Класс
StudentId,,
100024,Аникина София Константиновна,2a
100000,Беляев Роман Глебович,2a
100027,Воронова Анастасия Павловна,3b
100003,Евдокимов Дмитрий Дмитриевич,3b
100030,Жукова Александра Демидовна,2a
100033,Иванова Анна Сергеевна,3b
100006,Козырев Андрей Артёмович,2a
100036,Косарева Александра Марковна,2a
100009,Круглов Никита Серафимович,3b


## 2) Получим контактные данные родителей и учителя ученика


In [31]:
query = """Select \"STUDENTS\".\"StudentId\", \"STUDENTS\".\"StudentName\", \"STUDENTS\".\"Class\", \"STUDENTS\".\"ParentName\",
	\"STUDENTS\".\"ParentPhone\", \"TEACHERS\".\"TeacherName\", \"TEACHERS\".\"TeacherPhone\"
from \"STUDENTS\" join \"TEACHERS\" on \"STUDENTS\".\"Class\" = \"TEACHERS\".\"Class\"
Where \"STUDENTS\".\"StudentId\" = 100042"""

cursor.execute(query)
data = cursor.fetchall()
print(f"id: {data[0][0]} \nИмя, класс: {data[0][1]}, {data[0][2]}\nРодитель: {data[0][3]}, {data[0][4]}\nУчитель: {data[0][5]}, {data[0][6]}")

id: 100042 
Имя, класс: Павлова Мария Петровна, 2a
Родитель: Павлова Елизавета Артёмовна, 89995168153
Учитель: Агафонова Александра Ильинична, 89998683897


## 5) Составим расписание для преподавателя с id 502

In [55]:
query = """Select \"ADD_EDUCATION_TIME\".* from \"ADD_EDUCATION_TIME\"
where \"NameEdu\" in (select \"NameEdu\" from \"EDUCATORS\" where \"EducatorId\" = 502)
order by \"Time\""""

cursor.execute(query)
data = cursor.fetchall()
for dw in ['пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс']:
  print(f"{dw}: ")
  for i in range(len(data)):
    if data[i][1] == dw:
      print(f" {data[i][0]} {data[i][2]} Кабинет: {data[i][3]}")

пн: 
вт: 
ср: 
 Журналистика 16:30:00 Кабинет: 103
чт: 
пт: 
сб: 
 Журналистика 16:00:00 Кабинет: 103
вс: 
